# Parser Interface
This notebook illustrates how to great MDF-ready metadata with MDF-MatIO

In [1]:
from mdf_matio.adapters import noop_parsers
from mdf_matio import get_mdf_parsers, generate_search_index
from materials_io.utils import interface as matio
from tarfile import TarFile
import pandas as pd
import os

## Get the Available Parsers
The MDF only uses a limited subset of the data available via each parser.
Consequently, the MDF interface to MaterialsIO only uses parsers for which we have defined this desired subset.

In [2]:
all_parsers = matio.get_available_parsers()
print(f'Found {len(all_parsers)} parsers:', set(all_parsers.keys()))

c:\users\logan\documents\uc\citrine\software\materials-io\materials_io\file.py:12: UserWarning: The libmagic library is not installed. See: https://github.com/ahupp/python-magic#installation
  warn('The libmagic library is not installed. '


Found 7 parsers: {'em', 'crystal', 'dft', 'image', 'ase', 'generic', 'noop'}


One part of the MDF IO library is defining which parsers produce data in this format or a method for transforming the outputs of the data into a format compatible with the MDF's Search Index

In [3]:
print(f'Found {len(noop_parsers)} parsers that require zero alteration:', noop_parsers)

Found 2 parsers that require zero alteration: ['image', 'em']


In [4]:
mdf_parsers = get_mdf_parsers()
print(f'Found {len(mdf_parsers)} compatible parsers:', mdf_parsers)

Found 4 compatible parsers: {'em', 'image', 'generic', 'dft'}


Some of these parsers require an "adapter" to transform the data into the MDF format.

## Demonstrate Adapters
A good example of a parser that generates data in a non-MDF format is the "generic file parser."

In [5]:
test_file = os.path.join('example-files', 'dog2.jpeg')

In [6]:
generic_parser = matio.get_parser('generic')

The generic parser produces the hashes for the data file and, if installed, autodetects the file format.

In [7]:
file_info = generic_parser.parse([test_file])
file_info

{'length': 269360,
 'filename': 'dog2.jpeg',
 'path': 'example-files\\dog2.jpeg',
 'sha512': '1f47ed450ad23e92caf1a0e5307e2af9b13edcd7735ac9685c9f21c9faec62cb95892e890a73480b06189ed5b842d8b265c5e47cc6cf279d281270211cff8f90'}

The MDF search index stores this information under the the "files" block. 
Our adapter to the "generic" parser performs this operation.

In [8]:
generic_adapter = matio.get_adapter('generic')

In [9]:
generic_adapter.transform(file_info)

{'files': [{'length': 269360,
   'filename': 'dog2.jpeg',
   'path': 'example-files\\dog2.jpeg',
   'sha512': '1f47ed450ad23e92caf1a0e5307e2af9b13edcd7735ac9685c9f21c9faec62cb95892e890a73480b06189ed5b842d8b265c5e47cc6cf279d281270211cff8f90',
   'data_type': 'Unknown'}]}

The advantage of this adapter is that the MDF need not implement the hashing or file-type detection framework. 
The only tool needed for using this Materials IO parser is some data reshaping - a much easier task.

## Parsing MDF-Compliant Data
The `generate_search_index` function uses these capabilities to automatically generate compliant data from a directory of files.  It determines which parsers are available, runs them on all data in a directory, applies the adapters, and then merges the metadata of files records that describe the same record (e.g., a single experiment or calculation). 

Unpacking VASP data. (It is large enough that we do not want to commit the uncompressed files to GitHub).

In [10]:
with TarFile.open(os.path.join('example-files', 'calc', 'AlNi_static_LDA.tar.gz')) as t:
    t.extractall(os.path.join('example-files', 'calc'))

Deploying the search tool

In [11]:
record_gen = generate_search_index(os.path.join('example-files'), False)
record_gen

<generator object generate_search_index at 0x000002BCDA298CF0>

MaterialsIO uses generators to avoid needing to hold the entire dataset in memory at once.
Each metadata record is generated incremementally on-demand.

In [12]:
records = list(record_gen)
print(f'Generated {len(records)} records')

Generated 4 records


## Investigate Results of Parsing
There are 4 different records parsed from our example files.

For simplicity, we just print the paths of files associated with each record

In [13]:
def print_simple_paths(r):
    return [os.path.relpath(f['path']) for f in r['files']]

In [14]:
for i, r in enumerate(records):
    print(f'{i}:', print_simple_paths(r))

0: ['example-files\\group-by-dir\\dog1.jpeg', 'example-files\\group-by-dir\\dog2.jpeg', 'example-files\\group-by-dir\\mdf.json']
1: ['example-files\\dog2.jpeg']
2: ['example-files\\group-by-dir\\other-dir\\dog3.jpeg']
3: ['example-files\\calc\\AlNi_static_LDA\\CONTCAR', 'example-files\\calc\\AlNi_static_LDA\\DOSCAR', 'example-files\\calc\\AlNi_static_LDA\\INCAR', 'example-files\\calc\\AlNi_static_LDA\\KPOINTS', 'example-files\\calc\\AlNi_static_LDA\\OSZICAR', 'example-files\\calc\\AlNi_static_LDA\\OUTCAR', 'example-files\\calc\\AlNi_static_LDA\\POSCAR', 'example-files\\calc\\AlNi_static_LDA\\XDATCAR']


Our first record contains the several jpeg files that are grouped together.
Normally, they are JPEGs are not grouped together.
In this case the `mdf.json` file in that directory directs the parser to group records by directory.

In [15]:
with open(os.path.join('example-files', 'group-by-dir', 'mdf.json')) as fp:
    print(fp.read())

{
    "parse_by_directory": true
}


The configuration file applies to all subdirectories of `example-files/group-by-dir`

The other record that contains $>1$ file includes all of the files from a VASP calcualtion. 

In [16]:
dft_records = [x for x in records if 'dft' in x]

In [17]:
dft_records

[{'material': {'elemental_proportions': {'Al': 1}},
  'dft': {'converged': True,
   'exchange_correlation_functional': 'PAW',
   'cutoff_energy': 650.0},
  'origin': {'type': 'computation', 'name': 'VASP', 'version': '5.3.2'},
  'files': [{'length': 523,
    'filename': 'CONTCAR',
    'path': 'C:\\Users\\Logan\\Documents\\UC\\Citrine\\software\\mdf-matio\\notebooks\\example-files\\calc\\AlNi_static_LDA\\CONTCAR',
    'sha512': 'b29f1e6ced5bb681bf0ab19e264e0d182581fd8a84e47d20eb9c323791f0fe3e87a460c6c964dbcce971556c30860a4a141795c671ced3eb8c4d95528d128383',
    'data_type': 'Unknown'},
   {'length': 18297,
    'filename': 'DOSCAR',
    'path': 'C:\\Users\\Logan\\Documents\\UC\\Citrine\\software\\mdf-matio\\notebooks\\example-files\\calc\\AlNi_static_LDA\\DOSCAR',
    'sha512': 'defdb049f871123ae2fe93adc3e7a1834f236ac771f10e82b75bdc74f80eaffb7dfe395caeac2fc145df961bffddbabc1dd6e7c57399c7c03d94fb8e078ae2e7',
    'data_type': 'Unknown'},
   {'length': 460,
    'filename': 'INCAR',
    'pat

Note how this record contains >1 file and additional metadata for the DFT calculations